# Implementing BB84 protocol with cirq

This notebook is based on [lecture notes from TU Delft university](https://ocw.tudelft.nl/wp-content/uploads/LN_Week6.pdf)

## What is Quantum Key Distribution?

### First, private key cryptography

Imagine you've written a secret note that you only want your best friend to read. To keep it secret, you decide to scramble the letters in a specific way, a way that only you and your friend understand. This method of scrambling is your _key_. Now, even if someone else gets the note, they won't understand what it says because it's all scrambled up. Your friend, who knows the key, can unscramble the note to read the original message. This is the basic idea behind the private key, or symmetric, cryptography.

Private key cryptography is a type of encryption where the same key is used to both encrypt and decrypt the message. You use the key (a specific set of instructions) to convert your message into a secret code, and the same key is used to convert the secret code back into the original message. Because the same key is used for both processes, it's really important to keep the key secret. If someone else gets the key, they can decode all of your messages.

The main challenge in private key cryptography is securely sharing the key with the person you want to communicate with. If you're not careful, someone else might intercept the key and decode your messages. This is known as the _key distribution problem_.

### Quantum Key Distribution to the rescue!

Quantum Key Distribution (QKD) offers a solution to this problem using the principles of quantum mechanics. In the quantum world, measuring a system can change its state. This is called the _observer effect._ If someone tries to intercept the key and measures these photons, they will inevitably change their states. Your best friend will notice these changes when he receives the qubits, so they will know there was an eavesdropper. QKD solves the key distribution problem by enabling you and your friend to detect eavesdropping. If there's no eavesdropping, you can use the sequence of qubits that you've sent and your friend received to create a secret key. If there is eavesdropping, you discard the key and try again.

BB84 is one of the protocols realizing idea of QKD.

## Our battle plan and idea

In here we will not dive deep into mathematics (physics) behind every aspect of the protocol. I will provide you with references if you want to dive deeper. **Within scope of this notebook we are focusd on building the BB84 protocol using cirq, while allowing our solution not to be prefect, because we are learning.** 

With that preface our plan looks like this:
 1. Implement noise-free BB84 of the protocol
 2. Add some eavesdropping!
 3. Check how secure is the protocol.
 4. Add noise to the BB84 protocol.
 5. Double check how secure is the protocol.
 6. Use it to send secret message that is protected by law of physics.

## 1. Noise-free BB84

For the sake of the document sender will be called Alice and receiver will be called Bob. 

To implement noise-free BB84 we will follow this steps:
 1. Alice will generate two random binary (consisting only of `0`s and `1`s) strings - `alice_a, alice_b` - of the same length.
 2. Alice will use her strings to prepare a quantum state (sequence of qubits) that she will send to Bob.
 3. Bob will generate random binary string `bob_b` that he will use to try to _guess_ and decode Alice's message. Then Bob tells Alice that he received and measured the state he received.
 4. Alice and Bob exchange their `_b` strings. They discard all bits that do not match (from Alice's original message and Bob's guess-decoded message). They are left with `alice_kept_a` and `bob_kept_a` strings, respectively. 
 5. Alice picks a random set of indices `alice_kept_a` - `alice_test_indices` and sends them to Bob. Alice and Bob use `alice_test_indices` to create binary strings `alice_test_a` and `bob_test_a`. **If any of the bits miss-match they restart the procedure**
 6. They 

## Let's make some noise!

## Time to send a secret message